
# Speed Dating – Notebook d'analyse (squelette)

Ce notebook vous aide à explorer le dataset de **speed dating** (2002–2004) et à répondre à la question :  
**Quels facteurs influencent l'acceptation d'un second rendez-vous ?**

> **Livrables attendus** : statistiques descriptives, visualisations, et interprétations.


In [ ]:

# === Imports ===
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Règles graphiques du projet : utiliser matplotlib, un graphique par figure, pas de couleurs spécifiées
plt.rcParams['figure.figsize'] = (8, 5)
pd.set_option('display.max_columns', 120)



## 1. Chargement des données

- Indiquez le chemin vers votre fichier CSV (ou TSV).  
- Le dataset classique est souvent nommé `Speed Dating Data.csv` (ou similaire).  


In [ ]:

# === Paramètres utilisateur ===
DATA_PATH = os.getenv("SPEED_DATING_CSV", "speed_dating.csv")  # <-- modifiez si besoin

# === Chargement ===
# Si votre fichier est séparé par ';' ou '\t', adaptez le 'sep'.
try:
    df = pd.read_csv(DATA_PATH)
except Exception as e:
    print("Échec de lecture en CSV, tentative en TSV...")
    try:
        df = pd.read_csv(DATA_PATH, sep='\t')
    except Exception as e2:
        raise RuntimeError(f"Impossible de lire le fichier. Mettez à jour DATA_PATH. Détails: {e2}")

print(f"Shape: {df.shape}")
df.head()



## 2. Aperçu & dico des variables

- Regard sur la structure (`info`, `describe`, valeurs manquantes).
- Rappel des variables clés d'après le *data key* fourni.


In [1]:

# Vue d'ensemble
display(df.info())
display(df.describe(numeric_only=True).T.head(20))

# Taux de valeurs manquantes par variable
na_rate = df.isna().mean().sort_values(ascending=False)
na_rate.head(20)


NameError: name 'df' is not defined


### Variables clés (extrait du Data Key)

- **match** (1/0) : match réciproque (deux "Yes").  
- **dec** / **dec_o** : décision du participant / du partenaire ce soir-là (1=Yes, 0=No).  
- **attr, sinc, intel, fun, amb, shar** : notes attribuées au partenaire lors du *date* (1–10).  
- **attr_o, ...** : notes que **le partenaire** vous attribue.  
- **pf_o_att** : préférences déclarées du partenaire (vagues/échelles différentes selon *wave*).  
- **samerace** : 1 si même origine ethnique, 0 sinon.  
- **age, age_o** : âge du participant / du partenaire.  
- **gender** : 0 = Femme, 1 = Homme.  
- **order** : numéro du rendez-vous dans la soirée.  
- **attr3_1, ..., amb3_1** : **auto-évaluations** (Time 1).  
- **attr1_1, ..., shar1_1** : **importance déclarée** des attributs (Time 1).

> ⚠️ Les *waves* ont des échelles différentes (1–10 vs budget 100). Il faudra **normaliser** si on compare.



## 3. Préparation & nettoyage minimal

- Typage des variables (catégorielles vs numériques).  
- Harmonisation des variables d'importance (si comparaison inter-*waves*).  
- Création de variables dérivées utiles.


In [ ]:

# Copie de travail
data = df.copy()

# Harmonisation de quelques types
for col in ['match', 'dec', 'dec_o', 'samerace', 'gender']:
    if col in data.columns:
        data[col] = pd.to_numeric(data[col], errors='coerce')

# Sélection de colonnes d'intérêt disponibles
rating_cols = [c for c in ['attr','sinc','intel','fun','amb','shar'] if c in data.columns]
rating_o_cols = [c for c in ['attr_o','sinc_o','intel_o','fun_o','amb_o','shar_o'] if c in data.columns]

# Variable cible (ex. probabilité de "second date" côté participant)
target = 'dec' if 'dec' in data.columns else ('match' if 'match' in data.columns else None)
print("Cible choisie:", target)

# Filtre simple : on garde les lignes où la cible est connue
if target:
    data = data[~data[target].isna()].copy()

print("Shape après nettoyage minimal:", data.shape)



## 4. Statistiques descriptives

- Répartition par **genre** et **âge**.  
- Moyennes des **notes reçues** et **données**.  
- Taux de *yes* / *match* global et par genre.


In [ ]:

# Répartition par genre
if 'gender' in data.columns:
    print(data['gender'].value_counts(dropna=False))

# Statistiques d'âge
for c in ['age', 'age_o']:
    if c in data.columns:
        display(data[c].describe())

# Taux de "yes" (décision) et/ou "match"
for c in ['dec','dec_o','match']:
    if c in data.columns:
        rate = data[c].mean()
        print(f"Taux moyen {c}: {rate:.3f}")
        
# Moyennes des notes perçues des partenaires (si dispo)
if rating_o_cols:
    display(data[rating_o_cols].describe().T)

# Moyennes des notes données (si dispo)
if rating_cols:
    display(data[rating_cols].describe().T)



## 5. Visualisations

> Rappel : matplotlib uniquement, un graphique par figure, pas de couleurs spécifiées.

### 5.1 Distributions des notes (histogrammes)


In [ ]:

# Histogrammes des notes données au partenaire
for c in rating_cols:
    plt.figure()
    data[c].dropna().plot(kind='hist', bins=10, edgecolor='black')
    plt.title(f"Distribution de {c}")
    plt.xlabel(c)
    plt.ylabel('Fréquence')
    plt.show()



### 5.2 Comparaison des notes reçues par genre (boxplots)


In [ ]:

if 'gender' in data.columns and rating_o_cols:
    for c in rating_o_cols:
        plt.figure()
        # Boxplot simple par genre
        sub = data[['gender', c]].dropna()
        groups = [sub[sub['gender']==g][c].values for g in sorted(sub['gender'].dropna().unique())]
        plt.boxplot(groups, labels=[str(int(g)) for g in sorted(sub['gender'].dropna().unique())])
        plt.title(f"{c} reçu·e par genre (0=F, 1=H)")
        plt.xlabel('Genre')
        plt.ylabel(c)
        plt.show()



### 5.3 Corrélations avec la décision de second rendez-vous


In [ ]:

# Corrélations simples entre notes et décision
if target and (rating_cols + rating_o_cols):
    num_cols = [c for c in rating_cols + rating_o_cols if c in data.columns]
    corr = data[num_cols + [target]].corr(numeric_only=True)[target].sort_values(ascending=False)
    display(corr)
    
    plt.figure()
    corr.drop(labels=[target]).plot(kind='bar')
    plt.title(f"Corrélation (Pearson) avec {target}")
    plt.xlabel('Variables')
    plt.ylabel('Corrélation')
    plt.tight_layout()
    plt.show()



### 5.4 Shared interests vs Same race


In [ ]:

# Taux de "yes" selon intérêts partagés (approx via 'shar' élevé ?) vs 'samerace'
def rate_by_binary(col, threshold=None):
    if col not in data.columns:
        return None
    if threshold is not None:
        # Binarisation par seuil si note
        s = (data[col] >= threshold).astype(float)
    else:
        s = data[col]
    return data.groupby(s)[target].mean()

if target:
    # Même race
    if 'samerace' in data.columns:
        print("Taux par même origine ethnique (0/1):")
        print(data.groupby('samerace')[target].mean().round(3))

    # Intérêts partagés — proxy : note 'shar' donnée (>=7)
    if 'shar' in data.columns:
        print("\nTaux par 'shared interests' élevés (shar>=7):")
        res = rate_by_binary('shar', threshold=7)
        if res is not None:
            print(res.round(3))



### 5.5 Auto-perception vs perception réelle


In [ ]:

# Ex.: attractivité auto-évaluée (attr3_1) vs note reçue (attr_o)
x_col, y_col = 'attr3_1', 'attr_o'
if x_col in data.columns and y_col in data.columns:
    sub = data[[x_col, y_col]].dropna()
    if not sub.empty:
        plt.figure()
        plt.scatter(sub[x_col], sub[y_col], alpha=0.4)
        plt.title("Auto-perception vs perception réelle (Attractivité)")
        plt.xlabel("Auto-évaluation (attr3_1)")
        plt.ylabel("Note reçue (attr_o)")
        plt.show()
        corr_xy = sub.corr(numeric_only=True).loc[x_col, y_col]
        print(f"Corrélation {x_col} vs {y_col}: {corr_xy:.3f}")



### 5.6 Effet de l'ordre dans la soirée


In [ ]:

if target and 'order' in data.columns:
    order_rate = data.groupby('order')[target].mean()
    plt.figure()
    order_rate.plot(kind='line', marker='o')
    plt.title(f"Taux de {target} selon l'ordre du rendez-vous")
    plt.xlabel("Ordre dans la soirée")
    plt.ylabel(f"Taux de {target}")
    plt.tight_layout()
    plt.show()
    order_rate.head()



## 6. (Optionnel) Modèle explicatif simple

Un modèle logistique **interprétable** permet d'estimer l'effet marginal de quelques variables (contrôles simples).  
> ⚠️ Cette cellule suppose `scikit-learn` disponible. Sinon, laissez-la de côté.


In [ ]:

try:
    from sklearn.linear_model import LogisticRegression
    from sklearn.preprocessing import StandardScaler
    from sklearn.pipeline import Pipeline
except Exception as e:
    print("scikit-learn indisponible. Passez cette section si nécessaire.")
    LogisticRegression = None

if target and LogisticRegression is not None:
    features = [c for c in ['attr_o','sinc_o','intel_o','fun_o','amb_o','shar_o','age_o','samerace','order','gender'] if c in data.columns]
    sub = data.dropna(subset=[target] + features).copy()
    if not sub.empty:
        X = sub[features].values
        y = sub[target].values.astype(int)

        pipe = Pipeline([('scaler', StandardScaler(with_mean=True)), 
                         ('clf', LogisticRegression(max_iter=200, solver='lbfgs'))])
        pipe.fit(X, y)

        coefs = pipe.named_steps['clf'].coef_[0]
        coef_df = pd.DataFrame({'feature': features, 'coef': coefs}).sort_values('coef', ascending=False)
        display(coef_df)

        plt.figure()
        coef_df.set_index('feature')['coef'].plot(kind='bar')
        plt.title("Coefficients (log-odds) – modèle logistique")
        plt.xlabel('Variables')
        plt.ylabel('Coefficient')
        plt.tight_layout()
        plt.show()
    else:
        print("Échantillon vide après suppression des NA pour le modèle.")



## 7. Interprétation & insights

- **Quel(s) attribut(s)** ressort(ent) comme les plus liés au second rendez-vous ?  
- **Différences H/F** (si visibles).  
- **Intérêts partagés vs même origine** : lequel pèse le plus ?  
- **Auto-perception vs réel** : corrélation faible/forte ?  
- **Effet de l'ordre** : tendance notable ?

> **Implications pour Tinder** : comment améliorer les profils (photos, hobbies, mise en avant des centres d'intérêts), UX (ordre/rotation), et la personnalisation.



## 8. Conclusion

- Récapitulatif des insights clés.  
- Limites (biais d'échantillon, anciennes vagues, échelles différentes selon *wave*).  
- Ouverture (modèles prédictifs, tests de robustesse, segments d'utilisateurs).  
